In [1]:
# imports
import requests
import pandas as pd
import time
import datetime
import config

In [2]:
# your GraphQL API endpoint and headers (replace with your actual API key and URL)
SUBGRAPH_URL = config.SUBGRAPH_URL
headers = config.HEADERS
# amount to fetch with each request
BATCH_SIZE = 1000

In [ ]:
# function to fetch swaps with request to subgraph 

def fetch_swaps(pool_address, timestamp_gte=None, timestamp_lte=None, skip=0):
    filters = [f'pool: "{pool_address}"']
    # some defensive coding
    if timestamp_gte is not None:
        filters.append(f'timestamp_gte: {timestamp_gte}')
    if timestamp_lte is not None:
        filters.append(f'timestamp_lte: {timestamp_lte}')
    filters_str = ", ".join(filters)

# query for subgraph
    query = f"""
    {{
      swaps(first: {BATCH_SIZE}, skip: {skip}, orderBy: timestamp, orderDirection: asc, where: {{ {filters_str} }}) {{
        id
        timestamp
        sqrtPriceX96
        amount0
        amount1
        amountUSD
        tick
      }}
    }}
    """
    res = requests.post(SUBGRAPH_URL, json={"query": query}, headers=headers)
    res_json = res.json()
    if "errors" in res_json:
        raise Exception(res_json["errors"])
    return res_json["data"]["swaps"]

In [ ]:
# amount of trades is really large (for ETH/USDC), so we've added backup for the sake of not loosing progress if something goes 
# wrapper for download of swaps

def download_all_swaps(pool_address, timestamp_gte, timestamp_lte):
    all_swaps = []
    skip = 0
    try:
        while True:
            swaps = fetch_swaps(pool_address, timestamp_gte, timestamp_lte, skip)
            if not swaps:
                break
            all_swaps.extend(swaps)
            skip += BATCH_SIZE
            last_swap_time = datetime.datetime.utcfromtimestamp(int(swaps[-1]["timestamp"]))
            # to track progress
            print(f"Downloaded {len(all_swaps)} swaps so far... Last swap at: {last_swap_time} UTC")
            time.sleep(0.2)
    # if something goes wrong - save data
    except Exception as e:
        print("Error occurred while fetching swaps:", e)
        print("Saving progress to swaps_backup.csv")
        pd.DataFrame(all_swaps).to_csv("swaps_backup.csv", index=False)
        raise
    return all_swaps

In [ ]:

# fetching data for USDC/WETH
# pool address
POOL_ADDRESS = "0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640" # USDC/WETH 0.05%

if __name__ == "__main__":
    start_date = datetime.datetime(2024, 5, 1, 0, 0)
    end_date = datetime.datetime(2025, 5, 1, 0, 0)
    start_ts = int(start_date.timestamp())
    end_ts = int(end_date.timestamp())

    print(f"Downloading swaps from {start_date} to {end_date}...")

    swaps = download_all_swaps(POOL_ADDRESS, start_ts, end_ts)

    df = pd.DataFrame(swaps)


In [ ]:
# save data
df.to_csv("uniswap_v3_swaps_ETH_USDC_005.csv", index=False)
print(f"Saved {len(df)} swaps to uniswap_v3_swaps_ETH_USDC_005.csv")

In [ ]:
# fetching trades for USDC/USDT
# pool address
POOL_ADDRESS = "0x3416cf6c708da44db2624d63ea0aaef7113527c6"  # USDC/USDT 0.01%
# amount to fetch each time
BATCH_SIZE = 1000

if __name__ == "__main__":
    start_date = datetime.datetime(2024, 5, 1, 0, 0)
    end_date = datetime.datetime(2025, 5, 1, 0, 0)
    start_ts = int(start_date.timestamp())
    end_ts = int(end_date.timestamp())

    print(f"Downloading swaps from {start_date} to {end_date}...")

    swaps = download_all_swaps(POOL_ADDRESS, start_ts, end_ts)

    df = pd.DataFrame(swaps)



In [ ]:
# save data
df.to_csv("uniswap_v3_swaps_USDC_USDT_001.csv", index=False)

print(f"Saved {len(df)} swaps to uniswap_v3_swaps_USDC_USDT_001.csv")

In [ ]:
# fetching trades for PEPE/ETH
POOL_ADDRESS = "0x11950d141ecb863f01007add7d1a342041227b58"  # PEPE/ETH 0.3%
BATCH_SIZE = 1000

if __name__ == "__main__":
    start_date = datetime.datetime(2024, 5, 1, 0, 0)
    end_date = datetime.datetime(2025, 5, 1, 0, 0)
    start_ts = int(start_date.timestamp())
    end_ts = int(end_date.timestamp())

    print(f"Downloading swaps from {start_date} to {end_date}...")

    swaps = download_all_swaps(POOL_ADDRESS, start_ts, end_ts)

    df = pd.DataFrame(swaps)



In [ ]:
# save data
df.to_csv("uniswap_v3_swaps_PEPE_ETH_03.csv", index=False)

print(f"Saved {len(df)} swaps to uniswap_v3_swaps_PEPE_ETH_03.csv")